In [17]:
import numpy
import bempp.api
from bempp.api.operators.boundary import sparse, laplace
from scipy.sparse.linalg import gmres

In [18]:
def rhs_fun(x, n, domain_index,result):
    global phi_static
    result[:] = phi_static
    
def generate_grid(filename):
    
    face = open(filename+'.face','r').read()
    vert = open(filename+'.vert','r').read()

    faces = numpy.vstack(numpy.char.split(face.split('\n')[0:-1]))[:,:3].astype(int) - 1
    verts = numpy.vstack(numpy.char.split(vert.split('\n')[0:-1]))[:,:3].astype(float)

    grid = bempp.api.grid_from_element_data(verts.transpose(), faces.transpose())
    
    N = grid.leaf_view.entity_count(0)
    elements = list(grid.leaf_view.entity_iterator(0))
    area = numpy.zeros(N)

    # remove zero areas
    for i in range(N):
        area[i] = elements[i].geometry.volume

    area_nonzero = numpy.where(area>1e-12)[0]
    
    faces_nonzero = faces[area_nonzero,:]
    
    grid = bempp.api.grid_from_element_data(verts.transpose(), faces_nonzero.transpose())
    
    return grid

def energy_mol(mesh_diel, mesh_stern, phi_static, eps_s):

    grid_diel = generate_grid(mesh_diel)
    grid_stern = generate_grid(mesh_stern)

    space_diel = bempp.api.function_space(grid_diel, "DP", 0)
    space_stern = bempp.api.function_space(grid_stern, "DP", 0)
    
    N_diel = grid_diel.leaf_view.entity_count(0)
    N_stern = grid_stern.leaf_view.entity_count(0)
    
    elements_d = list(grid_diel.leaf_view.entity_iterator(0))
    elements_s = list(grid_stern.leaf_view.entity_iterator(0))

    phis_grid_fun = bempp.api.GridFunction(space_diel, fun=rhs_fun)

    rhs = numpy.concatenate([eps_s*phis_grid_fun.coefficients, 
                      numpy.zeros(N_stern)])
    

    M11   = laplace.single_layer(space_diel, space_diel, space_diel) 
    M12   = laplace.single_layer(space_stern, space_diel, space_diel) 
    M21   = laplace.single_layer(space_diel, space_stern, space_stern)
    M22   = laplace.single_layer(space_stern, space_stern, space_stern) 

    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0,0] = M11
    blocked[0,1] = M12
    blocked[1,0] = M21
    blocked[1,1] = M22
    op_discrete = blocked.strong_form()
    
    sigma, info = gmres(op_discrete, rhs, tol=1e-5, maxiter=500, restart = 1000)
    
    if info>0:
        print 'Not converged, %i iterations'%info
    elif info<0:
        print 'Solver diverges'
    
    sigma_d = sigma[:N_diel]
    sigma_s = sigma[N_diel:]

    
    qe = 1.60217662e-19
    Rw = 1.4
    Na = 6.0221409e23
    eps_0 = 8.854187817e-12
    
    #water_charge = 0.83*qe/(numpy.pi*Rw**2) * Na*qe*1e10/(1000*eps_0)
    water_dipole = 6.17e-30/(1e-10)/((2*Rw)**2) * Na*qe*1e10/(1000*eps_0)
    print water_dipole
    print sigma_s[:5]*1.4
    print max(abs(sigma_s*1.4))
    '''
    test_charge = numpy.where(abs(sigma_d)*1.4>water_dipole)[0]
    #print abs(sigma_d)
    if len(test_charge)>0:
        print 'Unphysical charge in dielectric surface'
        print len(test_charge)
    '''    
    
    test_charge = numpy.where(abs(sigma_s)*1.4>water_dipole)[0]
    if len(test_charge)>0:
        print 'Unphysical charge in stern surface'
        print len(test_charge)
    
    area_d = numpy.zeros(N_diel)
    for i in range(N_diel):
        area_d[i] = elements_d[i].geometry.volume
    
    energy = 0.5*numpy.sum(area_d*sigma_d*phi_static)

    return energy, float(len(test_charge))/N_stern

In [19]:
phi_static = 11. #kcal/mol/e
phi_static *= 4.184 # kJ/mol/e

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 5.5#80.

conv_factor = (1000*eps_0)/(qe**2*Na*1e10)

problem_folder = 'mobley_test/'
namefile = 'actually_all_names.txt'
num_lines = sum(1 for line in open(problem_folder+namefile))
energy = numpy.zeros(num_lines)
test_charge_arr = numpy.zeros(num_lines)
it = 0
for line in file(problem_folder+namefile):
    mol = line[:-1]

    mesh_stern = problem_folder+mol+'/surf_d02_stern'
    mesh_diel = problem_folder+mol+'/surf_d02'

    try: 
        e, test_charge = energy_mol(mesh_diel, mesh_stern, phi_static, eps_s)
    except:
        print 'Something went wrong'
        e = 0
        test_charge = 0.
            
    ekJ = e*conv_factor # kJ/mol
    ekcal = ekJ/4.184   # kcal/mol
    
    energy[it] = ekcal
    test_charge_arr[it] = test_charge
    
    print 'Run %i for '%it + mol + '. Energy = %1.3f kcal/mol'%energy[it]
    
    it += 1
    
numpy.savetxt('energy_bem.txt', energy)
numpy.savetxt('test_charge.txt', test_charge_arr)

857.593881087
[  129.80891032 -1955.16056699  -582.26452515  -693.71214179  -697.57311345]
2710.97629823
Unphysical charge in stern surface
63
Run 0 for 1112_tetrachloroethane. Energy = 11.089 kcal/mol
857.593881087
[ -571.24828932 -2242.66459034 -1033.28491285  -643.39357488  -795.01707029]
2647.14600776
Unphysical charge in stern surface
65
Run 1 for 111_trichloroethane. Energy = 10.262 kcal/mol
857.593881087
[  37.23922736 -411.37456286 -131.84941007  177.9957123   127.39727653]
5098.94976108
Unphysical charge in stern surface
89
Run 2 for 111_trifluoro_222_trimethoxyethane. Energy = 13.329 kcal/mol
857.593881087
[ -366.32904913 -2210.44196542 -1039.9632362   -710.58597641  -836.35496837]
2316.54608231
Unphysical charge in stern surface
53
Run 3 for 111_trifluoropropan_2_ol. Energy = 10.052 kcal/mol
857.593881087
[ 141.57902516 -120.02513267 -124.87152701  145.59010197  -62.04353283]
2672.86743511
Unphysical charge in stern surface
79
Run 4 for 111_trimethoxyethane. Energy = 12.537 

857.593881087
[  -91.77021653 -2248.34813048 -2314.08966831  -119.14311766  -404.86674409]
2825.3115685
Unphysical charge in stern surface
97
Run 42 for 1_bromohexane. Energy = 14.303 kcal/mol
857.593881087
[-370.61252866 -583.6927263  -715.09450398 -143.75556319 -294.25630907]
4365.59563591
Unphysical charge in stern surface
120
Run 43 for 1_bromooctane. Energy = 17.189 kcal/mol
857.593881087
[-347.13869356 -586.68973033 -415.40560383 -453.24297799 -462.5646011 ]
2611.89406452
Unphysical charge in stern surface
74
Run 44 for 1_bromopentane. Energy = 12.877 kcal/mol
857.593881087
[-330.25699207 -336.79193238 -202.13665487 -312.52937667 -167.16129316]
3779.65053431
Unphysical charge in stern surface
47
Run 45 for 1_bromopropane. Energy = 9.934 kcal/mol
857.593881087
[ -463.9659689  -1688.29491037  -672.98753731  -568.18735655  -864.72401918]
2098.67756382
Unphysical charge in stern surface
35
Run 46 for 1_chloro_222_trifluoroethane. Energy = 9.365 kcal/mol
857.593881087
[ -214.88675638 

857.593881087
[   6.0122459  -123.09007432   16.58068937  243.23982134 -419.57628417]
76196.9120854
Unphysical charge in stern surface
100
Run 84 for 25_dimethylphenol. Energy = 13.188 kcal/mol
857.593881087
[-313.66726092 -551.21612104 -524.63721301 -339.88833424 -521.06980974]
6207.9367124
Unphysical charge in stern surface
77
Run 85 for 25_dimethylpyridine. Energy = 12.551 kcal/mol
857.593881087
[-496.765848   -470.69512178 -567.44570697 -254.89434486 -278.5217155 ]
3755.3242154
Unphysical charge in stern surface
84
Run 86 for 25_dimethyltetrahydrofuran. Energy = 12.245 kcal/mol
857.593881087
[ -34.22334725  -21.81837192   -9.33576528 -138.6067878  -101.63238231]
41746.3761186
Unphysical charge in stern surface
115
Run 87 for 26_dimethylaniline. Energy = 13.190 kcal/mol
857.593881087
[ -371.43558377  -497.82015842  -240.92067399  -185.45573064 -1255.33377249]
44044.6050605
Unphysical charge in stern surface
171
Run 88 for 26_dimethylnaphthalene. Energy = 15.979 kcal/mol
857.59388108

857.593881087
[  195.23773849 -1739.74026945  -752.24834375  -620.25140802  -901.94112932]
3209.903557
Unphysical charge in stern surface
55
Run 126 for 2_methylpropan_1_ol. Energy = 10.182 kcal/mol
857.593881087
[ -321.38432147 -1636.79949796  -774.23413515  -807.63992748  -786.11614138]
6766.12477891
Unphysical charge in stern surface
66
Run 127 for 2_methylpropan_2_ol. Energy = 10.224 kcal/mol
857.593881087
[ -674.92737356  -987.09253934  -670.26083065  -785.74902651 -1891.69016139]
7917.29082838
Unphysical charge in stern surface
54
Run 128 for 2_methylpropane. Energy = 9.846 kcal/mol
857.593881087
[ -814.92396182 -1644.98404436  -670.00040842   422.58220912  -705.95953757]
10216.395117
Unphysical charge in stern surface
75
Run 129 for 2_methylpropene. Energy = 9.494 kcal/mol
857.593881087
[-308.84854631 -685.67438659 -471.92941213 -440.53214501 -603.51070999]
44515.4332884
Unphysical charge in stern surface
58
Run 130 for 2_methylpyrazine. Energy = 10.676 kcal/mol
857.593881087
[ 

857.593881087
[ -683.08316434  -964.49914858  -628.44070784  -822.91054007 -1718.99608571]
4003.86950858
Unphysical charge in stern surface
102
Run 168 for 3_methylheptane. Energy = 15.194 kcal/mol
857.593881087
[-250.48224705 -326.58983812 -353.93318389 -431.88783655 -445.22562948]
45325.5850324
Unphysical charge in stern surface
95
Run 169 for 3_methylhexane. Energy = 13.534 kcal/mol
857.593881087
[-211.9061536  -548.05097467 -197.58615311 -682.05801922 -485.4393434 ]
63831.0793057
Unphysical charge in stern surface
71
Run 170 for 3_methylpentane. Energy = 12.279 kcal/mol
857.593881087
[ 481.7069601  -568.64841836 -192.02030498  162.88586591 -104.91637265]
5740.65497124
Unphysical charge in stern surface
63
Run 171 for 3_methylpyridine. Energy = 11.029 kcal/mol
857.593881087
[ -587.2416264  -1151.17333722  -737.55626533  -829.03057888  -740.38251614]
55347.95214
Unphysical charge in stern surface
77
Run 172 for 3_nitroaniline. Energy = 12.259 kcal/mol
857.593881087
[ -848.26721811  -

...

857.593881087
[ 190.82911786 -745.47359357  -93.97688079  545.24782143   -4.13734245]
2404.11073426
Unphysical charge in stern surface
53
Run 297 for dimethoxymethane. Energy = 9.397 kcal/mol
857.593881087
[-913.8378046  -902.23585641 -770.76723522  -59.53018154 -901.75309177]
10247.7001299
Unphysical charge in stern surface
55
Run 298 for dimethyl_disulfide. Energy = 9.813 kcal/mol
857.593881087
[-498.6177213  -529.40886946 -426.64416239 -593.07813187 -931.41316647]
2088.77095589
Unphysical charge in stern surface
19
Run 299 for dimethyl_ether. Energy = 7.525 kcal/mol
857.593881087
[ 213.2640864   -42.7148533    85.66200211   80.68241602   79.99187645]
4070.53687902
Unphysical charge in stern surface
69
Run 300 for dimethyl_sulfate. Energy = 11.000 kcal/mol
857.593881087
[  31.04576426 -812.34890742  255.72676132  453.16688783  310.93661627]
15718.9159133
Unphysical charge in stern surface
34
Run 301 for dimethyl_sulfide. Energy = 8.277 kcal/mol
857.593881087
[ -847.28396214  -559.561

857.593881087
[-291.81696871 -255.84042524   13.81847919   66.5853826  -269.76582103]
351.158768465
Run 340 for hydrazine. Energy = 5.231 kcal/mol
Something went wrong
Run 341 for hydrogen_sulfide. Energy = 0.000 kcal/mol
857.593881087
[-302.50427144 -578.36360076 -502.67980831 -643.11032348 -664.31020644]
3411.48258082
Unphysical charge in stern surface
10
Run 342 for imidazole. Energy = 8.111 kcal/mol
857.593881087
[ -536.94130888 -1559.51417094  -826.44772751  -900.21511084  -666.09400173]
60353.7230961
Unphysical charge in stern surface
124
Run 343 for indane. Energy = 13.000 kcal/mol
857.593881087
[-860.19063154 -658.52829924 -909.061214   -494.91464072 -816.77605393]
40786.5370719
Unphysical charge in stern surface
111
Run 344 for iodobenzene. Energy = 11.813 kcal/mol
857.593881087
[-327.99999219 -518.94653221 -486.07881819 -387.00252203 -247.91844008]
2147.91663139
Unphysical charge in stern surface
33
Run 345 for iodoethane. Energy = 8.863 kcal/mol
857.593881087
[-573.47807959 

857.593881087
[ -846.40327794  -987.55086588  -312.71316699 -1098.08822936  -473.79660109]
54226.2726957
Unphysical charge in stern surface
108
Run 383 for methyl_p_nitrobenzoate. Energy = 15.151 kcal/mol
857.593881087
[-32134.37881824  30325.99498314   -294.9560045   -1805.22070242
   -652.8250596 ]
32134.3788182
Unphysical charge in stern surface
93
Run 384 for methyl_pentanoate. Energy = 13.763 kcal/mol
857.593881087
[ 716.52413655 -531.57857319  359.67949108 -258.91299163  184.62608394]
19596.3866139
Unphysical charge in stern surface
61
Run 385 for methyl_propanoate. Energy = 10.844 kcal/mol
857.593881087
[ -505.27395878  -935.51345363  -694.67547373  -731.40323789 -1528.26241297]
25446.9367676
Unphysical charge in stern surface
69
Run 386 for methyl_propyl_ether. Energy = 10.750 kcal/mol
857.593881087
[-1756.68078926  -223.28809173  -397.71895203 -1824.54152756  -931.18260421]
10349.0273903
Unphysical charge in stern surface
69
Run 387 for methyl_t_butyl_ether. Energy = 11.291 kc

857.593881087
[ 1127.19491564  -637.6588598     42.19840931  -162.16727468   107.77899422]
43445.5971876
Unphysical charge in stern surface
156
Run 425 for naphthalene. Energy = 13.156 kcal/mol
857.593881087
[-857.15536007 -658.17556561 -775.31701364 -572.0281939  -929.77316668]
56076.6827533
Unphysical charge in stern surface
90
Run 426 for nitrobenzene. Energy = 11.629 kcal/mol
857.593881087
[-278.82596116 -497.03481657 -318.81213097 -114.98783293 -278.22025869]
28663.1248837
Unphysical charge in stern surface
45
Run 427 for nitroethane. Energy = 8.833 kcal/mol
857.593881087
[ 247.8788734  -132.86833502 -160.46604557   60.31071191 -119.993186  ]
10489.2127073
Unphysical charge in stern surface
16
Run 428 for nitromethane. Energy = 7.312 kcal/mol
857.593881087
[-482.77276476 -407.80967106 -487.90265763 -333.599886   -500.88596867]
13343.3865779
Unphysical charge in stern surface
125
Run 429 for non_1_ene. Energy = 16.915 kcal/mol
857.593881087
[-1900.45639866 -2107.72845352  1357.8311

857.593881087
[-400.01067661 -511.01589402 -401.70359565 -452.78903596 -417.97826188]
29387.5011543
Unphysical charge in stern surface
35
Run 468 for propane. Energy = 8.648 kcal/mol
857.593881087
[ -291.71087795   443.86017146  -745.58360628 -1073.75913178  -645.38156548]
1864.82850845
Unphysical charge in stern surface
35
Run 469 for propanenitrile. Energy = 8.812 kcal/mol
857.593881087
[-107.53589565 -226.04667841 -341.06233413  142.10813592  120.5365408 ]
2670.22483226
Unphysical charge in stern surface
40
Run 470 for propanoic_acid. Energy = 9.121 kcal/mol
857.593881087
[ 202.44465831 -103.56767446 -268.30864209   73.05485052 -457.43925483]
15083.0539882
Unphysical charge in stern surface
49
Run 471 for propanone. Energy = 8.792 kcal/mol
857.593881087
[  627.60926275   892.27414177   144.6230064  -1347.25018077   -72.20459328]
2346.99097974
Unphysical charge in stern surface
31
Run 472 for propene. Energy = 8.155 kcal/mol
857.593881087
[ 329.07911851  383.67751307  338.44328072 -2

In [12]:
mesh_stern = 'mobley_test/nitroethane/surf_d02_stern'
mesh_diel = 'mobley_test/nitroethane/surf_d02'

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 5.5#80.

phi_static = 46. #kJ/mol/e
conv_factor = (1000*eps_0)/(qe**2*Na*1e10)


e, test_charge = energy_mol(mesh_diel, mesh_stern, phi_static, eps_s)
ekj = e * conv_factor#/ ((qe**2*Na*1e10)/(1000*eps_0))
ekcal = ekj/4.184
#print e, e / ((qe**2*Na*1e10)/(1000*eps_0)), e/(1000*4.184) #kJ/mol
print ekcal

857.593881087
[-278.6805626  -496.77562928 -318.64588094 -114.92787056 -278.07517599]
28648.1780082
Unphysical charge in stern surface
45
8.82394356584


In [33]:
grid_diel = generate_grid(mesh_diel)
grid_stern = generate_grid(mesh_stern)

space_diel = bempp.api.function_space(grid_diel, "DP", 0)
space_stern = bempp.api.function_space(grid_stern, "DP", 0)
    
N_diel = grid_diel.leaf_view.entity_count(0)
N_stern = grid_stern.leaf_view.entity_count(0)
    
elements_d = list(grid_diel.leaf_view.entity_iterator(0))
elements_s = list(grid_stern.leaf_view.entity_iterator(0))

print elements_d[0].geometry.corners
print elements_d[0].geometry.local2global(numpy.array([[1.,0.,0.]]))

[[-0.802 -0.885 -0.46 ]
 [ 1.119  0.807  1.406]
 [ 0.339  0.758  0.614]]


AttributeError: 'Geometry' object has no attribute 'global2local'

In [20]:
F = 1000*eps_0/(qe**2*Na*1e10)
1/(4*numpy.pi*F*4.184)
9081457.48434/(332.063*4*numpy.pi)
(qe**2*Na*1e10)/(eps_0)
2353.39653991/40

58.834913497749994

In [ ]:
1.54